Funciona aqui pq eu preciso relembrar C, mas estudar como fazer um único código no C pra dar a saida direto

In [1]:
import numpy as np
import pandas as pd

In [2]:
ano_i = 2000.00   # ano inicial
ano_f = 2020.00   # ano final

lat_i = -89.00    # latitude inicial  (-90 dá erro)
lat_f = 89.00    # latitude final  (90 dá erro)

lon_i = -180.00   # lonfitude inicial
lon_f = 180.00   # longitude final

d_ano = 10.0      # intervalo entre anos
d_coord = 2.0     # intervalo entre coordenadas

alt = 0.0        # altitude

In [3]:
# ok

lats = np.arange(lat_i, lat_f+d_coord, d_coord)
lats = lats.reshape(len(lats),1)
lons = np.arange(lon_i, lon_f+d_coord, d_coord)
lons = lons.reshape(len(lons),1)
anos = np.arange(ano_i, ano_f+d_ano, d_ano)
anos = anos.reshape(len(anos),1)

#coluna de longitudes ok
lons_2 = np.repeat(lons, len(lats), axis=1).reshape(len(lons)*len(lats), 1)
lons_3 = np.repeat(lons_2, len(anos), axis=1)
lons_3 = lons_3.T.reshape(lons_3.size, 1)

# coluna de latitudes ok
lats_2 = np.repeat(lats, len(lons), axis=1).T.reshape(len(lons)*len(lats), 1)
lats_3 = np.repeat(lats_2, len(anos), axis=1)
lats_3 = lats_3.T.reshape(lats_3.size, 1)

# coluna de anos ok
anos_2 = np.repeat(anos, len(lons_2), axis=1).reshape(len(lons)*len(lats)*len(anos), 1)

# strings extras para padrão de coordenadas (em graus) e altitude (em metros)
extras = [['D', 'M'+str(alt)]]*len(lons)*len(lats)*len(anos)

In [4]:
# Padrão do input:     2019.5 D M0 70.3 30.8
M = np.concatenate((anos_2, extras, lats_3, lons_3), axis=1)
np.savetxt('my_input.txt', M, delimiter=" ", fmt="%s")

In [5]:
# calcula o IGRF usando o Geomag 7.0
!geomag70.exe IGRF12.COF f my_input.txt my_output.txt



 Geomag v7.0 - Jan 25, 2010 

 'f' switch: converting file with multiple locations.
     The first five output columns repeat the input coordinates.
     Then follows D, I, H, X, Y, Z, and F.
     Finally the SV: dD, dI, dH, dX, dY, dZ,  and dF
     The units are the same as when the program is
     run in command line or interactive mode.


 Processed 48870 lines




log: DOMAIN error

log: DOMAIN error

log: DOMAIN error

log: DOMAIN error

log: DOMAIN error

log: DOMAIN error

log: DOMAIN error

log: DOMAIN error


In [6]:
a = pd.read_csv('my_output.txt', header=0, delim_whitespace=True)

# parece que funcionou - duplica os NaN da coluna D_deg
mask = a.D_deg.isna()
a.loc[mask, 'D_deg':] = a.loc[mask, 'D_deg':].shift(periods=1, axis=1)

# parece que funcionou - duplica os NaN da coluna I_deg
mask = a.I_deg.isna()
a.loc[mask, 'I_deg':] = a.loc[mask, 'I_deg':].shift(periods=1, axis=1)

In [7]:
Date = a['Date'].to_numpy(dtype=int)
Latitude = a['Latitude'].to_numpy(dtype=int)
Longitude = a['Longitude'].to_numpy(dtype=int)

F_nT = a['F_nT'].to_numpy(dtype=float)

# pega a parte numérica dos graus e dos minutos, excluindo a notação d e m
# dos valores presentes nestas colunas
D_deg = a['D_deg'].to_list()
D_deg_num = np.array([float(str(i).split('d')[0]) for i in D_deg])
D_min = a['D_min'].to_list()
D_min_num = np.array([float(str(i).split('m')[0]) for i in D_min])

I_deg = a['I_deg'].to_list()
I_deg_num = np.array([float(str(i).split('d')[0]) for i in I_deg])
I_min = a['I_min'].to_list()
I_min_num = np.array([float(str(i).split('m')[0]) for i in I_min])


# converte de graus e minutos para graus decimais
sinal = np.sign(D_deg_num)
sinal[sinal==0] = np.sign(D_min_num[sinal==0])
D_graus = sinal * (np.abs(D_deg_num) + np.abs(D_min_num)/60.0)

sinal = np.sign(I_deg_num) 
sinal[sinal==0] = np.sign(I_min_num[sinal==0])
I_graus = sinal * (np.abs(I_deg_num) + np.abs(I_min_num)/60.0)

In [8]:
#D_deg_num[-2]
D_min_num[-2]

22.0

In [9]:
n = 315      # indice para teste

print('I_deg_num =', I_deg_num[n])
print('I_min_num =', I_min_num[n])
print('I_graus =', I_graus[n])

I_deg_num = 0.0
I_min_num = -25.0
I_graus = -0.4166666666666667


In [10]:
print(np.where(I_deg_num<0))
print(np.where(I_min_num<0))

(array([    0,     1,     2, ..., 48823, 48824, 48825], dtype=int64),)
(array([  315,  1034,  1124,  1214,  1933,  2023,  2113,  2742,  2832,
        3281,  3371,  3730,  4089,  4179,  4628,  4718,  4808,  4898,
        4988,  5349,  5892,  6164,  6617,  7069,  7610,  7700,  7790,
        8330,  8420,  8510,  8600,  8690,  8780,  9679,  9769, 14898,
       14988, 15527, 15617, 15976, 16605, 16695, 17414, 17504, 18223,
       18313, 18942, 19032, 19571, 20020, 20379, 20469, 21459, 21730,
       21911, 22183, 22455, 22998, 23630, 23720, 24890, 24980, 25070,
       25879, 26598, 26688, 26778, 26868, 26958, 27048, 27138, 27228,
       27318, 27408, 27498, 27588, 27678, 29568, 29658, 29748, 29838,
       29928, 30018, 31008, 31098, 31188, 31278, 31817, 31907, 32356,
       32895, 32985, 33614, 33704, 34423, 34513, 35142, 35232, 35861,
       36310, 36759, 37569, 37840, 38021, 38474, 38746, 38927, 39108,
       39379, 39740, 41360, 41450, 42079, 42708, 42798, 42888, 44058,
       44148, 4423

# Lê o arquivo calculado


D_deg, D_min, I_deg, I_min = np.loadtxt('my_output.txt', skiprows=1, usecols=(5,6,7,8), dtype=str, unpack=True)
Date, Latitude, Longitude, F_nT, dD_min, dI_min, dF_nT = np.loadtxt('my_output.txt', skiprows=1, 
                                                                    usecols=(0, 3,4,13,14,15,20), dtype=float, unpack=True)

D_deg_num = np.array([float(i.split('d')[0]) for i in D_deg])
D_min_num = np.array([float(i.split('m')[0]) for i in D_min])
I_deg_num = np.array([float(i.split('d')[0]) for i in I_deg])
I_min_num = np.array([float(i.split('m')[0]) for i in I_min])


sinal = np.sign(D_deg_num)
sinal[sinal==0] = 1
D_graus = sinal * (np.abs(D_deg_num) + D_min_num/60.0)

sinal = np.sign(I_deg_num)
sinal[sinal==0] = 1
I_graus = sinal * (np.abs(I_deg_num) + I_min_num/60.0)

dD_graus = dD_min/60.0
dI_graus = dI_min/60.0

In [11]:
# salva o arquivo em formato de tabela
IGRF = np.concatenate((Date.reshape(Date.size, 1), \
                       Latitude.reshape(Latitude.size, 1), \
                       Longitude.reshape(Longitude.size, 1), \
                       F_nT.reshape(F_nT.size, 1), \
                       # dF_nT.reshape(dF_nT.size, 1), \
                       D_graus.reshape(D_graus.size, 1), \
                       # dD_graus.reshape(dD_graus.size, 1), \
                       I_graus.reshape(I_graus.size, 1)), \
                       # dI_graus.reshape(dI_graus.size, 1)), \
                      axis=1)

np.savetxt('IGRF.txt', IGRF, fmt=['%d', '%d', '%d', '%.4f', '%.4f', '%.4f'])

In [12]:
# apaga os outros arquivos
#!del my_input.txt
#!del my_output.txt